In [7]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
from preprocessing import (
    clsf_boz_cd_c, clsf_crd_grd, get_disc_factor_all_c, get_loss_adj_rate_all_c
)

In [2]:
# 환경설정
pd.options.display.float_format = '{:,.0f}'.format
os.makedirs('result', exist_ok=True)

# 전역변수
FILE_PATH = Path('./data/현행추정부채_자동차')
BASE_YYMM = '201912'

In [18]:
자동차_비례출재_미경과보험료

,BSC_CVR_CD,PDGR_CD,PDC_CD,INER_CHN_DVCD,USTK_DVCD,RN_LTPD_URND_PRM
0,04,08,20001,03,1,0
1,01,08,20014,01,1,0
2,02,09,20016,99,1,266217364
3,06,09,20016,99,1,298099310
4,01,09,20023,01,1,465095371
...,...,...,...,...,...,...
77,05,09,20016,04,1,5041256
78,01,09,20016,04,1,622660350
79,09,09,20016,03,1,2187916
80,02,09,20016,10,1,399979


In [17]:
# 데이터 불러오기
## 보험료부채 익스포져
자동차_원수_미경과보험료 = pd.read_excel(FILE_PATH / f'자동차_원수_미경과보험료_{BASE_YYMM}.xlsx', dtype={'BSC_CVR_CD': str, 'PDGR_CD': str, 'PDC_CD': str, 'INER_CHN_DVCD': str, 'USTK_DVCD': str})
자동차_비례출재_미경과보험료 = pd.read_excel(FILE_PATH / f'자동차_비례출재_미경과보험료_{BASE_YYMM}.xlsx', dtype={'BSC_CVR_CD': str, 'PDGR_CD': str, 'PDC_CD': str, 'INER_CHN_DVCD': str, 'USTK_DVCD': str})
자동차_비비례출재_미경과보험료 = pd.read_excel(FILE_PATH / f'자동차_비비례출재_미경과보험료_{BASE_YYMM}.xlsx', dtype={'BSC_CVR_CD': str, 'PDGR_CD': str, 'PDC_CD': str, 'SL_CVR_CD': str, 'T02_RN_RINSC_CD': str})
# 일반_원수_예정보험료 = pd.read_excel(FILE_PATH / f'일반_원수_예정보험료_{BASE_YYMM}.xlsx', dtype={'RRNR_DAT_DVCD': str, 'RRNR_CTC_BZ_DVCD': str, 'ARC_INPL_CD': str})
# 일반_출재_예정보험료 = pd.read_excel(FILE_PATH / f'일반_출재_예정보험료_{BASE_YYMM}.xlsx', dtype={'RRNR_DAT_DVCD': str, 'RRNR_CTC_BZ_DVCD': str, 'ARC_INPL_CD': str, 'T02_RN_RINSC_CD': str})
자동차_원수_직전3년보험금손조비 = pd.read_excel(FILE_PATH / f'자동차_원수_직전3년보험금손조비_{BASE_YYMM}.xlsx', dtype={'BSC_CVR_CD': str, 'PDGR_CD': str, 'PDC_CD': str, 'INER_CHN_DVCD': str})
자동차_비례출재_직전3년보험금손조비 = pd.read_excel(FILE_PATH / f'자동차_비례출재_직전3년보험금손조비_{BASE_YYMM}.xlsx', dtype={'BSC_CVR_CD': str, 'PDGR_CD': str, 'PDC_CD': str, 'INER_CHN_DVCD': str})
자동차_비비례출재_직전3년보험금손조비 = pd.read_excel(FILE_PATH / f'자동차_비비례출재_직전3년보험금손조비_{BASE_YYMM}.xlsx', dtype={'PDC_CD': str, 'T02_RN_RINSC_CD': str})

## 준비금부채 익스포져
자동차_원수_개별추산액 = pd.read_excel(FILE_PATH / f'자동차_원수_개별추산액_{BASE_YYMM}.xlsx', dtype={'BSC_CVR_CD': str, 'PDGR_CD': str, 'PDC_CD': str, 'INER_CHN_DVCD': str})
자동차_비례출재_개별추산액 = pd.read_excel(FILE_PATH / f'자동차_비례출재_개별추산액_{BASE_YYMM}.xlsx', dtype={'BSC_CVR_CD': str, 'PDGR_CD': str, 'PDC_CD': str, 'INER_CHN_DVCD': str})
자동차_비비례출재_개별추산액 = pd.read_excel(FILE_PATH / f'자동차_비비례출재_개별추산액_{BASE_YYMM}.xlsx', dtype={'T02_RN_RINSC_CD': str, 'PDC_CD': str})

## 기초정보, 가정 등
# 자동차_코드맵핑 = pd.read_excel(FILE_PATH / '자동차_코드맵핑.xlsx', 
#     dtype={'PDGR_CD': str, 'BSC_CVR_CD': str, 'INER_CHN_DVCD': str, 'USTK_DVCD': str, 'AT_PDGR_CD': str, 'AT_BSC_CVR_CD': str, 'AT_CHN_DVCD': str})
# 자동차_코드맵핑.to_csv(FILE_PATH / '자동차_코드맵핑.csv', index=False)
자동차_코드맵핑 = pd.read_csv(FILE_PATH / '자동차_코드맵핑.csv',
    dtype={'PDGR_CD': str, 'BSC_CVR_CD': str, 'INER_CHN_DVCD': str, 'USTK_DVCD': str, 'AT_PDGR_CD': str, 'AT_BSC_CVR_CD': str, 'AT_CHN_DVCD': str})
자동차_상품군정보 = pd.read_excel(FILE_PATH / '자동차_상품군정보.xlsx', dtype={'PDGR_CD': str, 'BSC_CVR_CD': str})
자동차_보험금진전추이 = pd.read_excel(FILE_PATH / f'자동차_보험금진전추이_{201812}.xlsx', dtype={'PDGR_CD': str, 'AY': str}) \
        .rename(columns={'AT_PDGR_CD': 'BOZ_CD', 'AY_YM': 'AY'})
자동차_최종손해율 = pd.read_excel(FILE_PATH / f'자동차_최종손해율_{201812}.xlsx', dtype={'RRNR_DVCD': str, 'AT_CHN_DVCD': str, 'AT_PDGR_CD': str})
자동차_사업비율 = pd.read_excel(FILE_PATH / f'자동차_사업비율_{BASE_YYMM}.xlsx', dtype={'RRNR_DVCD': str, 'BOZ_CD': str})
자동차_IBNR = pd.read_excel(FILE_PATH / f'자동차_IBNR_{BASE_YYMM}.xlsx', dtype={'RRNR_DVCD': str, 'BSC_CVR_CD': str})

## 공통
재보험자_국내신용등급 = pd.read_excel(FILE_PATH / f'재보험자_국내신용등급_{202012}.xlsx', dtype={'재보험사코드': str}) \
        .rename(columns = {'재보험사코드': 'T02_RN_RINSC_CD', '국내신용등급': 'CRD_GRD'})
선도부도율 = pd.read_excel(FILE_PATH / f'선도부도율_{202012}.xlsx')
할인율 = pd.read_excel(FILE_PATH / f'할인율_{202012}.xlsx')

In [4]:
# 데이터 전처리
## 미경과보험료
자동차_원수_미경과보험료_가공 = 자동차_원수_미경과보험료.copy()
자동차_원수_미경과보험료_가공['RRNR_DVCD'] = '04'
자동차_원수_미경과보험료_가공['DMFR_DVCD'] = '01'
자동차_원수_미경과보험료_가공['BOZ_CD'] = clsf_boz_cd_c(자동차_원수_미경과보험료_가공, 자동차_상품군정보)

자동차_비례출재_미경과보험료_가공 = 자동차_비례출재_미경과보험료.copy()
자동차_비례출재_미경과보험료_가공['RRNR_DVCD'] = '03'
자동차_비례출재_미경과보험료_가공['DMFR_DVCD'] = '01'
자동차_비례출재_미경과보험료_가공['BOZ_CD'] = clsf_boz_cd_c(자동차_비례출재_미경과보험료_가공, 자동차_상품군정보)
자동차_비례출재_미경과보험료_가공['T02_RN_RINSC_CD'] = '124012'
자동차_비례출재_미경과보험료_가공['CRD_GRD'] = clsf_crd_grd(자동차_비례출재_미경과보험료_가공, 재보험자_국내신용등급)

자동차_비비례출재_미경과보험료_가공 = 자동차_비비례출재_미경과보험료.copy()
자동차_비비례출재_미경과보험료_가공['RRNR_DVCD'] = '03'
자동차_비비례출재_미경과보험료_가공['DMFR_DVCD'] = '01'
자동차_비비례출재_미경과보험료_가공['BOZ_CD'] = clsf_boz_cd_c(자동차_비비례출재_미경과보험료_가공, 자동차_상품군정보)
자동차_비비례출재_미경과보험료_가공['CRD_GRD'] = clsf_crd_grd(자동차_비비례출재_미경과보험료_가공, 재보험자_국내신용등급)

## 직전3년보험금손조비
자동차_원수_직전3년보험금손조비_가공 = 자동차_원수_직전3년보험금손조비.copy()
자동차_원수_직전3년보험금손조비_가공['RRNR_DVCD'] = '04'
자동차_원수_직전3년보험금손조비_가공['DMFR_DVCD'] = '01'
자동차_원수_직전3년보험금손조비_가공['BOZ_CD'] = clsf_boz_cd_c(자동차_원수_직전3년보험금손조비_가공, 자동차_상품군정보)

자동차_비례출재_직전3년보험금손조비_가공 = 자동차_비례출재_직전3년보험금손조비.copy()
자동차_비례출재_직전3년보험금손조비_가공['RRNR_DVCD'] = '04'
자동차_비례출재_직전3년보험금손조비_가공['DMFR_DVCD'] = '01'
자동차_비례출재_직전3년보험금손조비_가공['BOZ_CD'] = clsf_boz_cd_c(자동차_비례출재_직전3년보험금손조비_가공, 자동차_상품군정보)
자동차_비례출재_직전3년보험금손조비_가공['T02_RN_RINSC_CD'] = '124012'
자동차_비례출재_직전3년보험금손조비_가공['CRD_GRD'] = clsf_crd_grd(자동차_비례출재_직전3년보험금손조비_가공, 재보험자_국내신용등급)

자동차_비비례출재_직전3년보험금손조비_가공 = 자동차_비비례출재_직전3년보험금손조비.copy()
자동차_비비례출재_직전3년보험금손조비_가공['RRNR_DVCD'] = '04'
자동차_비비례출재_직전3년보험금손조비_가공['DMFR_DVCD'] = '01'
# 자동차_비비례출재_직전3년보험금손조비_가공['BOZ_CD'] = clsf_boz_cd_c(자동차_비비례출재_직전3년보험금손조비_가공, 자동차_상품군정보)
자동차_비비례출재_직전3년보험금손조비_가공['CRD_GRD'] = clsf_crd_grd(자동차_비비례출재_직전3년보험금손조비_가공, 재보험자_국내신용등급)

## 개별추산액
자동차_원수_개별추산액_가공 = 자동차_원수_개별추산액.copy()
자동차_원수_개별추산액_가공['RRNR_DVCD'] = '04'
자동차_원수_개별추산액_가공['DMFR_DVCD'] = '01'
자동차_원수_개별추산액_가공['BOZ_CD'] = clsf_boz_cd_c(자동차_원수_개별추산액_가공, 자동차_상품군정보)

자동차_비례출재_개별추산액_가공 = 자동차_비례출재_개별추산액.copy()
자동차_비례출재_개별추산액_가공['RRNR_DVCD'] = '04'
자동차_비례출재_개별추산액_가공['DMFR_DVCD'] = '01'
자동차_비례출재_개별추산액_가공['BOZ_CD'] = clsf_boz_cd_c(자동차_비례출재_개별추산액_가공, 자동차_상품군정보)
자동차_비례출재_개별추산액_가공['T02_RN_RINSC_CD'] = '124012'
자동차_비례출재_개별추산액_가공['CRD_GRD'] = clsf_crd_grd(자동차_비례출재_개별추산액_가공, 재보험자_국내신용등급)

자동차_비비례출재_개별추산액_가공 = 자동차_비비례출재_개별추산액.copy()
자동차_비비례출재_개별추산액_가공['RRNR_DVCD'] = '04'
자동차_비비례출재_개별추산액_가공['DMFR_DVCD'] = '01'
# 자동차_비비례출재_개별추산액_가공['BOZ_CD'] = clsf_boz_cd_c(자동차_비비례출재_개별추산액_가공, 자동차_상품군정보)
자동차_비비례출재_개별추산액_가공['CRD_GRD'] = clsf_crd_grd(자동차_비비례출재_개별추산액_가공, 재보험자_국내신용등급)

# 자동차_개별추산액_가공 = pd.concat([자동차_원수_개별추산액_가공, 자동차_비례출재_개별추산액_가공, 자동차_비비례출재_개별추산액_가공], axis=0).reset_index(drop=True)

In [13]:
# 가정산출

## 최종손해율
자동차_최종손해율['RRNR_DVCD'] = '01'

## 할인요소
자동차_할인요소 = get_disc_factor_all_c(자동차_보험금진전추이, 할인율.query('KICS_SCEN_NO == 1'))

## 재보험자산 손실조정율
손실조정율 = get_loss_adj_rate_all_c(자동차_보험금진전추이, 할인율.query('KICS_SCEN_NO == 1'), 선도부도율)